# Module 1: Standardizing Missing Data

## Part 1: Searching for Missing Data

Data cleaning and missing data go hand-in-hand. Unidentified missing values will persist unnoticed and subsequently lead to issues during later stages of the analysis. These values could either be invalid, such as a value of 15 in an integer column with a valid range of 1 to 10, or they might be formatted in a way that doesn't appropriately signal their absence.

Handling "out of range" values in a numeric series is a straightforward process. All that's required is to obtain the minimum and maximum values of the column and then replace any values that fall outside this valid range with a designated missing data indicator, such as `np.nan`.

Identifying improperly formatted missing values can be more challenging, especially when dealing with columns that contain strings. Missing values formatted as strings can manifest in various forms (e.g., `"-999"`, `"--"`). Dealing with such cases often requires creative approaches or versatile tools like regular expressions.

### Guided Task

Consider a research project focused on comparing salaries across European countries. Data were gathered using an online questionnaire, making it accessible to a wide range of individuals. The eligibility criteria encompassed being an employed adult residing in a European country. 

Our task is to explore the "Age" and "Country" variables for any inconsistencies and address them. The csv file for this task can be found in the notes of this video. Load it up and let's go.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("Data/EuroSalaries.csv")
df.head()

,Age,Country,Salary
0,26,Sweden,30635.0
1,66,Cyprus,29612.0
2,66,United Kingdom,97246.0
3,24,Andorra,40195.0
4,51,Germany,87278.0


Given that the research project focuses solely on adults, any age values below 18 are considered invalid. We should also set an upper limit such as 100 or 120, considering that life expectancy rarely extends beyond that range.

Subsequently, it becomes essential to devise a strategy for handling out-of-range age values. These anomalies may arise from data entry errors or technical glitches, but they might also represent genuine entries. If we identify them as errors, marking them as missing data is a viable solution. Conversely, if these values are genuine, we should consider removing the entire row from the dataset. This is because individuals falling beyond the specified age range are outside the study's intended focus.

In [2]:
# Get min and max age
print("Minimum age:", df["Age"].min())
print("Maximum age:", df["Age"].max())

# Display a sample of ages above 100 yrs old
print(
    df.loc[df["Age"] > 100, "Age"].sample(5)
)

# Mark invalid ages as missing
df.loc[df["Age"] > 100, "Age"] = np.nan

Minimum age: 19
Maximum age: 170
3049    125
3047    162
2111    151
224     144
1122    162
Name: Age, dtype: int64


It is very unlikely that values above 100 are genuine and that is why they are marked as missing.

Now that we have made the necessary adjustments to the "Age" column, we should also have a look at its data type. This will let us know of the values it can contain so we should expect a numeric data type.

In [3]:
df["Age"].dtype

dtype('float64')

Being float is actually what we would expect after introducing `np.nan` into the column. `np.nan` is underneath a floating-point value and will cause an integer column to convert into float. We will see how to deal with this issue in part 2 of this course.

The research project has the goal of comparing salaries among European countries. Consequently, the "Country" column should exclusively contain entries of European countries. When dealing with categorical variables like this, there are two approaches to identify problematic data:

The first approach involves utilizing the .unique() method to display all countries within the column. However, this method can be challenging to implement if the variable contains an extensive list of categories.

The second approach is to validate entries in the "Country" column by comparing them to a pre-established list of legitimate European countries. Such a list can be sourced externally from reliable sources such as Google or by consulting resources like chatGPT. 

In [4]:
# Option A: Manually identify errors
df["Country"].unique()

# Option B: Compare values in Country to a list of valid values (generated by chatGPT)
european_countries = ["Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", 
                      "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", 
                      "France", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", 
                      "Latvia", "Liechtenstein", "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", 
                      "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", 
                      "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", 
                      "Switzerland", "Ukraine", "United Kingdom", "Vatican City"]

## Unlike lists, sets contain only unique values - handy when printing data
potential_errors = set()
for country in df["Country"].to_numpy():
    if country not in european_countries:
        potential_errors.add(country)
print(potential_errors)

{'Canada', 'India', '--'}


Two countries that are not located in Europe, namely India and Canada, were identified within the "Country" column. Additionally, an entry with an invalid value was found.

To ensure the accuracy and focus of our study, we should proceed by eliminating rows containing non-European countries from the dataset. Their data does not align with the scope of our investigation.

Concerning the entry with an invalid value, it's recommended to flag that specific instance as missing.

In [5]:
# Drop rows where the value in the 'Country' column is 'India' or 'Canada'
countries_to_drop = ['India', 'Canada']
df = df[~df['Country'].isin(countries_to_drop)]

# Replace '--' with np.nan to indicate missingness
df["Country"] = df["Country"].replace("--", np.nan)

A data type check is not necessary for the "Country" column. Unlike "Age", which contains only numerical values, the "Country" column exclusively holds strings and is thus classified as an "object" column type. Instead of focusing on data types, it would be more beneficial to concentrate on identifying non-string values. This is because an "object" column can accommodate values of various data types.

However, we have already addressed this while searching for errors in the column. If there was a value outside of the `european_countries` list present within the column, it would have been displayed in the console.

In practice, it's advisable to thoroughly understand your data before carrying out any operations on it. In the context of this course, we've swiftly explored error identification and correction. However, when dealing with real-world data, it's essential to dedicate more time to comprehending the diverse column data types as this understanding guides us in determining the appropriate operations to apply.

## Part 2: Missing Data Formats

It is essential that the format of missing data being used can be recognized by our chosen data analysis library, which in our case is pandas. Popular examples include:

- <strong>np.nan</strong> (NaN): This is the representation of a floating-point "Not-a-Number" value in NumPy. It's commonly used to represent missing or undefined values in numeric arrays. 
  
- <strong>pd.NaT</strong> (NaT): "Not a Time" is specific to datetime columns in pandas.
  
- <strong>None</strong>: This is a special object in Python that also represents missing values.

np.nan was preferred for representing missing values in pandas due to its alignment with the NumPy library. Since pandas was developed on the foundation of NumPy, which had already established the convention of utilizing np.nan to denote missing or undefined numerical values, this choice became a natural fit.

However, limitations arose in terms of type compatibility. We have already seen how the mere presence of np.nan led to the conversion of integer values into floats. Another issue occurs when attempting to convert all values to the string data type. This process results in missing values being changed into strings, thereby removing their original indicator that denoted their absence. To address these limitations and others, pandas v.1.0 introduced `pd.NA`:

- <strong>pd.NA</strong> (\<NA\>): A specialized missing value marker that works with various data types.

Unlike `np.nan` which is an actual value, `pd.NA` acts as a missing indicator which means that it does not affect column data types and other values, but rather it is ignored by most calculations. 

Let's dive into the EuroSalaries dataset to test it. The "Salary" column should be an integer, but it's currently a float due to missing values formatted as np.nan. We'll see if replacing np.nan with pd.NA changes the column's data type to integer.

In [6]:
df["Salary"] = df["Salary"].replace(np.nan, pd.NA)
df["Salary"].dtype

dtype('O')

The data type did change, but not as anticipated. Instead of becoming an integer, it shifted to an object type. Within standard pandas data types, pd.NA doesn't yield the expected results and shares comparable limitations with using np.nan.  In version 1.0 of pandas, they also introduced nullable data types that are capable of accommodating missing values effectively.  

## Part 3: Nullable Data Types

Nullable data types can accommodate missing values without altering the state of the data. They were designed to improve memory use and performance while offering a user-friendly approach to manage missing data more flexibly. 


| Standard Pandas Dtype | Corresponding Nullable Dtype|
|------------------|-----------------------------|
| int64            | Int64                       |
| float64          | Float64                     |
| bool             | BooleanDtype                |
| datetime64       | No direct nullable type     |
| timedelta64      | No direct nullable type     |
| object           | StringDtype                 |
| category         | No direct nullable type     |

Regular pandas data types can be converted to their nullable counterpart with the `.astype()` command:

- Nullable integer: `df[col].astype("Int64")` - note the capital `I`
- Nullable float: `df[col].astype("Float64")` - note the capital `F`
- Nullable boolean: `df[col].astype(pd.BooleanDtype())` 
- Nullable string: `df[col].astype(pd.StringDtype())` or `df[col].astype("string")`

We can also automatically convert all columns with the `.convert_dtypes()` method. Let's see it in action with the EuroSalaries dataset:

In [7]:
df_nullable = df.convert_dtypes()
df_nullable.dtypes

Age         Int64
Country    string
Salary     object
dtype: object

All columns have transitioned to nullable data types. For instance, the "Age" column, previously holding float data due to np.nan values, now holds integers. This change occurred because, when converting columns to nullable, all missing value representations are unified to pd.NA. Since pd.NA functions well with various data types, the "Age" column appropriately assumed its true data type.

If you wish to learn more about conventional data types and the new additions, you can check out the notes associated with this video.

---

<u>Video Notes</u>

<h4>Pandas Data Types</h4>

Pandas provides various data structures, including different data types for handling different types of data. Here is a list of pandas data types along with a brief description for each:

- object: This allows for mixed data types within a column although it is often used for string data.

- int8, int16, int32, int64: These are integer data types and are used for columns that contain whole numbers.

- float16, float32, float64: These are floating-point data types and are used for columns that contain decimal numbers.
 
- bool: This represents a boolean data type, which can have values True or False. It's often used for columns that contain binary or categorical data.

- datetime64: This represents date and time values with nanosecond precision. It's used for columns that store timestamp information.

- timedelta64: This represents differences between two datetime64 values. It's used for columns that store durations or time intervals.

- category: This represents categorical data. It's used to store data with a limited and fixed set of values.

<br>
New additions:

- StringDtype: This is an extension of the object data type for columns that contain only strings. It supports both strings and missing values.
  
- Int8, Int16, Int32, Int64: These are nullable integer data types that can support both integers and missing values.

- UInt8, UInt16, UInt32, UInt64: These are nullable unsigned integer data types. They're used for columns where only positive values are needed and memory usage is a concern.

- Float16, Float32, Float64: These are nullable floating-point data types that can support both floats and missing values.

- PeriodDtype: This is used for representing regular intervals of time, such as days, months, or years.

- BooleanDtype: This is a nullable boolean data type. It's introduced to address scenarios where you want to represent missing or unknown boolean values in your data.

- CategoricalDtype: Similar to the category data type, this is used for representing categorical data, but it allows for custom ordering of categories.

- SparseDtype: This is used to represent sparse data structures, which efficiently store data with a lot of missing values.

<h4>Memory Efficiency and Computational Speed</h4>

You might have noticed that data types like int or float offer various bit sizes (e.g. `int8`, `int16`, `int32`, `int64`). These choices are designed to cater to diverse needs related to memory capacity and computation speed. Smaller bit sizes in data types mean they are less precise, but they consume less memory and lead to quicker calculations. Precision refers to how accurately a number can be represented. Just like a ruler with more marks can measure more precisely, higher precision data types can capture finer details in numbers.

For instance, when we convert a column to the `int` type using a command like `.astype(int)`, the default allocation is 64 bits. This allows the column to hold very large whole numbers with a high level of accuracy. However, in some situations, such as when working with smaller numbers, we might not need such high precision. In such cases, opting for a smaller data type like `int32` could be more memory-efficient without sacrificing accuracy.

In essence, allocating more bits leads to greater precision but lower efficiency in terms of memory and speed.

---

## Exercise

We have seen how to handle invalid entries and explored efficient ways of formatting data. To practice what we have learned, we will use the data of 30000 runners. Our goal is to spot and rectify incorrect values, ensure consistent column data types, and ensure that all missing values can be retrieved with the .isna() method. 

The exercise requires a few coding procedures that were not covered in the course. These were intended to encourage you to use the web as you would do in a real-world setting. If you find yourself struggling, you can check out the hints in the notes of this video.

Good luck!


---

### Hints

- The dataset has a total of 5131 missing values
- Regular expression site for testing and cheatsheet: https://pythex.org/

### Column-specific Hints

- In the "Experience" column the are two invalid instances. These can be obtained with the `.unique()` method and adjusted to your liking with `.replace()`. 

- "Sleep Duration (hours)", "Distance Covered (km)"	and "Average Pace (min/km)" have invalid values. These can be located with the `str.contains()` method along with a regular expression like `[^\d\.]+`. In cases where `.replace()` does not work as expected, try using `str.replace()` instead.

- The column "Average Heart Rate (bpm)" is already in numeric format but includes values that are out of range. Heart rate values above 220 are too extreme for human physiology. You may consider replacing these with `pd.NA` by following this template: `df.loc[df[condition, "column"] = value_to_assign`

---

## Solution

We will now go through the exercise together. If you didn't manage to get the expected results then don't worry. Remember that working with data requires a lot of patience and practice.

Let's get started.

My initial step is to display a sample of the data and get an understanding of how the majority of values within each column look like.

In [1]:
import pandas as pd

df = pd.read_csv("Data/running_data.csv")
df.head()

,Experience,Sleep Duration (hours),Distance Covered (km),Average Pace (min/km),Average Heart Rate (bpm)
0,advanced,7.0,12.02,4.59,119
1,beginner,6.0,3.19,6.9,200
2,advanced,8.0,19.57,4.73,112
3,advanced,7.0,10.27,3.3,117
4,beginner,6.0,7.66,6.02,180


I then like to examine data types and check if the values in columns match the expected data type. 

In [2]:
df.dtypes

Experience                  object
Sleep Duration (hours)      object
Distance Covered (km)       object
Average Pace (min/km)       object
Average Heart Rate (bpm)     int64
dtype: object

From these, I can already tell that most of the columns have problematic values. For example, columns like sleep duration, distance covered, and average pace contain floats, yet their data type is listed as object. This means that other data types could exist within them. 

In contrast, experience and average heart rate are not so obvious because they hold a reasonable data type according to their values.

Now is probably a good point to convert columns to their nullable data type, so I wil just go ahead and do that.

In [3]:
df = df.convert_dtypes()
df.dtypes

Experience                  string
Sleep Duration (hours)      string
Distance Covered (km)       string
Average Pace (min/km)       string
Average Heart Rate (bpm)     Int64
dtype: object

Let's just take these columns one at a time and start with experience.
Experience looks like a categorical variable with few categories. We may be able to locate errors by displaying all its unique values.

In [4]:
df["Experience"].unique()

<StringArray>
['advanced', 'beginner', 'intermediate', '   ', 'begin']
Length: 5, dtype: string

Notice that the "Experience" category comprises three main labels: "advanced," "beginner," and "intermediate". However, we can see two additional values: an empty string and "begin". It's likely that "begin" stands for "beginner".

To address this, we can treat the empty string as missing data and use a regular expression that matches white space to capture it. As for the value "begin", we can adjust it to "beginner" to align with the existing three categories.

In [5]:
df["Experience"] = df["Experience"].replace(r"^\s*$", pd.NA, regex=True)
df["Experience"] = df["Experience"].replace("begin", "beginner")

# Check
df["Experience"].unique()

<StringArray>
['advanced', 'beginner', 'intermediate', <NA>]
Length: 4, dtype: string

Moving on. 

Sleep duration, distance covered, and average pace were initially labeled as type "object," which led to their data type becoming "string" after the conversion. However, based on the extracted sample, they should have likely been represented as floats. This discrepancy suggests that values might have been mistakenly formatted as strings, or there might be a mix of different data types within these columns.

We can start by looking for non-numeric values. The `str.contains()` method can be used to create a mask based on a regular expression pattern which in our case will capture non-numeric values. This mask will then allow us to display these values and decide if there are any issues in the column.

In [6]:
# Create mask for non-numeric values using regex
non_numeric_mask = df["Sleep Duration (hours)"].str.contains(r"[^\d\.]+", regex=True)

# Display values
df.loc[non_numeric_mask, "Sleep Duration (hours)"]

647      ?
859      ?
949      ?
961      ?
1171     ?
        ..
28393    ?
28486    ?
29456    ?
29656    ?
29830    ?
Name: Sleep Duration (hours), Length: 66, dtype: string

It appears that the problematic value is a question mark which is probably supposed to indicate missingness. Let's replace all occurences of this value with a valid missing indicator:

In [7]:
df["Sleep Duration (hours)"] = df["Sleep Duration (hours)"].replace("?", pd.NA)

If no other data errors are present, the sleep duration column should now be able to be converted into a numeric type. Once this is done, we should have a look at the range of values to ensure no inconsistencies

In [8]:
df["Sleep Duration (hours)"] = df["Sleep Duration (hours)"].astype("Float64")

# Get min and max values
print(df["Sleep Duration (hours)"].min())
print(df["Sleep Duration (hours)"].max())

6.0
8.0


Next up are the columns distance covered and average pace. These are similar in nature to sleep duration, so we can repeat the same procedure.

In [9]:
non_numeric_mask = df["Distance Covered (km)"].str.contains(r"[^\d\.]+", regex=True)
df.loc[non_numeric_mask, "Distance Covered (km)"]

9                      a17.9
20                     a7.95
27                    a18.81
29       a16.490000000000002
42                    a12.75
                ...         
29968                 a18.17
29982                 a10.28
29985                 a19.67
29990                  a9.63
29991                  a9.78
Name: Distance Covered (km), Length: 3900, dtype: string

In this scenario, certain values are preceded by the letter "a". To revert them to their float type, we can simply remove the letter. This can be achieved by slicing and re-assigning the values after the "a" to their original positions.

In [10]:
df.loc[non_numeric_mask, "Distance Covered (km)"] = df["Distance Covered (km)"].str[1:]

Let's make sure that the changes took place using an example with the value at index 9.

In [11]:
# Test with an example 
df.loc[df["Distance Covered (km)"].index == 9, "Distance Covered (km)"]

9    17.9
Name: Distance Covered (km), dtype: string

Letter "a" is removed, meaning that the values are corrected. What remains is to convert the column to a numeric type and getting its range.

In [12]:
df["Distance Covered (km)"] = df["Distance Covered (km)"].astype("Float64")
print(df["Distance Covered (km)"].min())
print(df["Distance Covered (km)"].max())

3.0
20.5


The minimum and maximum values look reasonable so we can move on to the next column. 
The majority of values for average pace look like floats. 

In [13]:
non_numeric_mask = df["Average Pace (min/km)"].str.contains(r"[^\d\.]+", regex=True)
df.loc[non_numeric_mask, "Average Pace (min/km)"]

57       6,19
112      6,76
126      4,45
319      5,17
587      5,26
         ... 
29179    5,78
29516    5,06
29548    3,94
29550    5,47
29594    6,98
Name: Average Pace (min/km), Length: 303, dtype: string

Using the same approach as before, we discover that specific values are using a comma as a decimal separator. This can be easily resolved by substituting these commas with periods. However, using `.replace()` alone won't suffice here.

In a previous instance, we employed the `.replace()` method to substitute question marks with `pd.NA` in the "Sleep Duration" column like this: `df["Sleep Duration (hours)"].replace("?", pd.NA)`. This approach worked as we were replacing entire strings with another value.

In the current task, the comma acts as a substring within a larger string. To achieve the desired outcome, we must utilize the `str` accessor in conjunction with the `replace` method. This combination enables us to replace substrings with a different character.

In [14]:
df["Average Pace (min/km)"] = df["Average Pace (min/km)"].str.replace(",", ".")

All occurences of a comma should now be replaced with a dot. If values are consistent we should now be able to convert to float.

In [15]:
df["Average Pace (min/km)"] = df["Average Pace (min/km)"].astype("Float64")

ValueError: could not convert string to float: ' . '

The conversion was not sucessful. The error informs us that a particular value `' . '` cannot be converted. We must find these invalid values and flag them as missing before attempting to convert the column again.

In [16]:
non_numeric_mask = df["Average Pace (min/km)"].str.contains(r"[^\d\.]+", regex=True)

# Flag invalids
df.loc[non_numeric_mask, "Average Pace (min/km)"] = pd.NA

# Convert to float
df["Average Pace (min/km)"] = df["Average Pace (min/km)"].astype("Float64")

Finally, let's have a look at the minimum and maximum values

In [17]:
print(df["Average Pace (min/km)"].min())
print(df["Average Pace (min/km)"].max())

3.0
7.0


Average pace is ready for analysis. The last column we should look at is average heart rate. This is correctly formatted as numeric so all we need to do is look for any values that do not make sense for a variable like heart rate.

In [18]:
print(df["Average Heart Rate (bpm)"].min())
print(df["Average Heart Rate (bpm)"].max())

90
362


The minimum value looks reasonable, but the maximum value is not something that a human can achieve. A quick search into google informs us that the maximum heart rate of any person can be roughly estimated by substracting age from 220. Therefore, we can be sure that any value above 220 is inaccurate. 

Such values can be marked as missing.

In [19]:
df.loc[df["Average Heart Rate (bpm)"] > 220, "Average Heart Rate (bpm)"] = pd.NA

# Check
print(df["Average Heart Rate (bpm)"].max())

218


The maximum heart rate observed is now 218. We could argue that is too high but I would say it is within the realm of possibility when considering individual variations and situations of extreme stress.

It's crucial to emphasize that establishing an upper limit beyond which values are deemed invalid is a distinct procedure from addressing outliers. Outliers represent values that are unusually extreme but still feasible within a given context. However, in our specific scenario, any heart rate exceeding 220 is considered invalid due to its inherent impossibility within the range of human physiological capabilities.

This is all we need to do to have this data ready for analysis. All column data types have been standardized, values are accurate and all missing values are correctly formatted.

In [20]:
df.isna().sum()

Experience                   292
Sleep Duration (hours)       336
Distance Covered (km)          0
Average Pace (min/km)          3
Average Heart Rate (bpm)    4500
dtype: int64

See you in the next video!